In [1]:
import pandas as pd
import os

folder = "../datasets"

list_df = []

for file_name in os.listdir(folder):
    if file_name.endswith('.csv'):  
        filepath = os.path.join(folder, file_name)
        df = pd.read_csv(filepath)
        list_df.append(df)

final_df = pd.concat(list_df, ignore_index=True)

final_df.drop_duplicates(subset=['id'], keep='first', inplace=True)

final_df.to_csv("../data/final_dataset.csv", index=False)


/var/folders/bv/_91mjjdj0n3djcbgvr1bxlph0000gn/T/ipykernel_23109/2331666755.py:11: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [2]:
import pandas as pd 
import re
import numpy as np

In [3]:
data = pd.read_csv('../data/final_dataset.csv', index_col='id', dtype={4: str, 17: str})

In [4]:
data.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
id,,,,,,,,,,,,,,,,,
31840,Serviced apartment in Florence · ★4.65 · 1 bed...,380378,Officina 360,NaN,Centro Storico,43.77709,11.25216,Private room,191,1,128,2023-09-04,0.79,30,335,6,NaN
32120,Rental unit in Florence · ★4.88 · 1 bedroom · ...,99235,Lucia,NaN,Gavinana Galluzzo,43.76157,11.27741,Entire home/apt,95,1,26,2023-07-17,0.16,1,52,6,NaN
32180,Condo in Florence · ★4.77 · 2 bedrooms · 4 bed...,13925330,Cesare,NaN,Centro Storico,43.76832,11.24348,Entire home/apt,105,6,31,2023-08-05,0.19,2,275,8,NaN
32395,Bed and breakfast in Florence · ★4.84 · 1 bedr...,140170,Fabio,NaN,Centro Storico,43.76268,11.24149,Hotel room,70,1,662,2023-08-28,4.15,5,9,76,06139100488
39115,Home in Florence · ★4.56 · 1 bedroom · 2 beds ...,167739,Lorenzo,NaN,Centro Storico,43.77525,11.25258,Private room,89,2,70,2023-07-24,0.44,11,276,12,NaN


In [5]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 237258 entries, 31840 to 973895808066047620
Data columns (total 17 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   name                            237258 non-null  object 
 1   host_id                         237258 non-null  int64  
 2   host_name                       237233 non-null  object 
 3   neighbourhood_group             53825 non-null   object 
 4   neighbourhood                   237258 non-null  object 
 5   latitude                        237258 non-null  float64
 6   longitude                       237258 non-null  float64
 7   room_type                       237258 non-null  object 
 8   price                           237258 non-null  int64  
 9   minimum_nights                  237258 non-null  int64  
 10  number_of_reviews               237258 non-null  int64  
 11  last_review                     187183 non-null  object 
 12  revie

None

We observe here that the variables 'neighborhood_group' and 'license' contain a large number of NaN. If we refer to the dictionary associated with this dataset, we can note that these variables:
- License corresponds to The license/permit/registration number
- neighborhood_group corresponds to The neighborhood as geocoded using the latitude and longitude against neighborhoods as defined by open or public digital shapefiles.

This is not necessarily data that I consider interesting. Indeed, License varies depending on the legislation in force in the country and/or city and here, we have taken cities in different countries. As for the neighborhood_group, we don't necessarily need it because we have all the lagitudes and longitudes.

So I'm going to purely eliminate these variables from our dataset.

Regarding, 'last_review' and 'reviews_per_month' there are exactly the same number of NaNs.
So I will check that number_of_reviews = 0 for the goods concerned.

In [6]:
# Removal of variables considered as unnecessary
data = data.drop(['neighbourhood_group', 'license'], axis=1)

In [7]:
# Filter rows where 'last_review' is NaN
filtered_data = data[pd.isna(data['last_review'])]
display(filtered_data.head())

# Check if 'number_of_reviews' is equal to 0 for these rows
check_reviews = all(filtered_data['number_of_reviews'] == 0)

if check_reviews:
    print("All rows with 'last_review' as NaN also have 'number_of_reviews' equal to 0.")
else:
    print("Some rows with 'last_review' as NaN have 'number_of_reviews' different from 0.")


,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
id,,,,,,,,,,,,,,,
82496,Rental unit in Florence · 2 bedrooms · 4 beds ...,434231,Magda,Centro Storico,43.77550,11.25826,Entire home/apt,140,6,0,NaN,NaN,3,303,0
523048,Rental unit in Florence · 1 bedroom · 2 beds ·...,2377476,Clara E Carlo,Centro Storico,43.75932,11.24215,Entire home/apt,170,3,0,NaN,NaN,12,316,0
523221,Rental unit in Florence · 2 bedrooms · 3 beds ...,2377476,Clara E Carlo,Gavinana Galluzzo,43.73593,11.24707,Entire home/apt,85,5,0,NaN,NaN,12,0,0
531903,Rental unit in Florence · 1 bedroom · 1 bed · ...,2377476,Clara E Carlo,Gavinana Galluzzo,43.75237,11.23798,Entire home/apt,91,15,0,NaN,NaN,12,99,0
750529,Bed and breakfast in Florence · 1 bedroom · 1 ...,1372403,Donatella,Centro Storico,43.77678,11.24045,Private room,100,2,0,NaN,NaN,3,131,0


All rows with 'last_review' as NaN also have 'number_of_reviews' equal to 0.


In [8]:
display(data.describe())

,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,2.372580e+05,237258.000000,237258.000000,237258.000000,237258.000000,237258.000000,187183.000000,237258.000000,237258.000000,237258.000000
mean,1.536548e+08,48.431300,1.536041,185.603900,25.019662,30.301958,1.227290,17.647557,128.664559,8.681119
std,1.692109e+08,4.360532,5.056745,658.366673,79.276148,67.268025,1.592554,58.282394,133.102637,17.614696
min,2.750000e+02,38.676140,-9.488100,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,1.772050e+07,48.838030,-0.144920,70.000000,1.000000,1.000000,0.210000,1.000000,0.000000,0.000000
50%,6.947939e+07,48.893084,2.165730,117.000000,3.000000,6.000000,0.690000,1.000000,77.000000,2.000000
75%,2.638260e+08,51.519080,2.365620,198.000000,5.000000,27.000000,1.690000,6.000000,260.000000,10.000000
max,5.369828e+08,52.656110,13.721390,95195.000000,1125.000000,3199.000000,120.110000,595.000000,365.000000,1689.000000


By observing the DataFrame, we realize that the name variable contains additional information about the accommodations and I will therefore extract this information to create new variables.

In [9]:
# Number of available bedrooms - if not listed in the 'name' column, it is considered to have no bedroom
data['Bedrooms'] = data['name'].str.extract(r'(\d+\.?\d*)\s+(?:bedrooms|bedroom)', flags=re.IGNORECASE).fillna('0')
data['Bedrooms'] = data['Bedrooms'].astype(float)

In [10]:
# Number of available beds - if not listed in the 'name' column, it is considered to have no bed
data['Beds'] = data['name'].str.extract(r'(\d+\.?\d*)\s+(?:beds|bed)', flags=re.IGNORECASE).fillna('0')
data['Beds'] = data['Beds'].astype(float)

In [11]:
# Number of available baths - if not listed in the 'name' column, it is considered to have no bath
data['Baths'] = data['name'].str.extract(r'(\d+\.?\d*|Half)\s+(?:private\s+)?baths?', flags=re.IGNORECASE).fillna('0')

# Replace 'Half' with 0.5
data['Baths'] = data['Baths'].replace('Half', '0.5')
data['Baths'] = data['Baths'].astype(float)

In [12]:
# Extract the rating from the 'name' column, if there is none, replace with '0'.

data['Rating'] = data['name'].str.extract(r'\★(\d+\.\d+)', flags=re.IGNORECASE).fillna('0')

data['Rating'] = data['Rating'].astype(float)


In [13]:
# Conversion of 'last_review' to datetime
data.loc[:, 'last_review'] = pd.to_datetime(data['last_review'], errors='coerce') 

In [14]:
# Extract Housing Types from the 'name' column
data['Housing types'] = data['name'].str.extract(r'^(.*?) in', flags=re.IGNORECASE)

print(data['Housing types'].unique())

['Serviced apartment' 'Rental unit' 'Condo' 'Bed and breakfast' 'Home'
 'Villa' 'Loft' 'Boutique hotel' 'Guest suite' 'Townhouse' 'Vacation home'
 'Cottage' 'Place to stay' 'Guesthouse' 'Farm stay' 'Tiny home'
 'Aparthotel' 'Hotel' 'Resort' 'Cabin' 'Bungalow' 'Dome' 'Tent' 'Hostel'
 'Earthen home' 'casa particular' 'Castle' 'Camper/RV' 'Tower' 'Boat'
 'Houseboat' 'Barn' 'Nature lodge' 'Chalet' 'Cave' 'Yurt' 'minsu' 'Floor'
 'Windmill' 'Lighthouse' 'Shipping container' 'Hut' 'Campsite' 'pension'
 'Treehouse' 'Bus' 'Holiday park' 'Tipi' 'Island' 'Ice dome' 'trullo'
 "shepherd's hut" 'Religious building' 'Riad']


In [15]:
# Cluster creation based on Housing Type

clusters = {
    'Private Residential Properties': [
        'Home', 'Villa', 'Townhouse', 'Cottage', 'Bungalow', 'Cabin', 
        'Chalet', 'Castle', 'Tower'
    ],
    'Private Urban Accommodations': [
        'Apartment', 'Condo', 'Loft', 'Serviced apartment', 'Aparthotel', 'Rental unit',
        'Riad', 'Religious building'
    ],
    'Compact and Efficient': [
        'Tiny home', 'Dome', 'Shipping container', 'Hut', 'Windmill', 'Lighthouse'
    ],
    'Rural Accommodations': [
        'Farm stay', 'Nature lodge', 'Campsite', 'Eco lodge', 'Holiday park', 'Island',
        'Barn', 'Treehouse'
    ],
    'Specialty Accommodations': [
        'Cave', 'Ice dome', 'trullo', "Shepherd's hut", 'Earthen home', 'Bus', 
        'Tipi', 'Yurt', 'minsu', 'Floor', 'Tent'
    ],
    'Mobile Accommodations': [
        'Camper/RV', 'Boat', 'Houseboat'
    ],
    'Short-Term Rentals': [
        'Bed and breakfast', 'Hostel', 'Pension', 'Guesthouse', 'Guest suite'
    ],
    'Hotels': [
        'Hotel', 'Resort', 'Boutique hotel'
    ]
}

def get_property_type_cluster(housing_types):
    for cluster_name, types in clusters.items():
        if housing_types in types:
            return cluster_name
    return 'Other'  

data['Property Type Cluster'] = data['Housing types'].apply(get_property_type_cluster)

To extract cities, it's slightly more complex. Indeed, sometimes it is indeed the city in 'name' but mostly it is something else such as the postal code for example.

I will extract the name of the cities corresponding to my initial datasets when they are present in name. If this is not the case I retrieve the name of the cities from a mapping carried out from neighborhood which does not contain any null value.

In [16]:
# Extract the location from the 'name' column
data['Location'] = data['name'].str.extract(r'\bin\b\s+(.+?)\s+·', flags=re.IGNORECASE).fillna('Unknown')
data['Location'] = data['Location'].str.title()

known_cities = ['London', 'Paris', 'Amsterdam', 'Berlin', 'Brussels', 'Florence', 'Lisbon', 'Barcelona']

# Set to 'Unknown' if the extracted city is not in the list of known cities
data['Location'] = data['Location'].apply(lambda x: 'Unknown' if x not in known_cities else x)

# Initialize the dictionary for mapping
neighbourhood_to_city = {}

# Update the dictionary with existing data
for index, row in data.iterrows():
    city = row['Location']
    neighbourhood = row['neighbourhood']
    
    if city in known_cities:
        neighbourhood_to_city[neighbourhood] = city

# Use the dictionary to fill in missing values
data['Location'] = data.apply(
    lambda row: neighbourhood_to_city.get(row['neighbourhood'], 'Unknown') if row['Location'] not in known_cities else row['Location'],
    axis=1
)

In [17]:
unknown_count = (data['Location'] == 'Unknown').sum()
unique_neighbourhood_count = data[data['Location'] == 'Unknown']['neighbourhood'].nunique()
print('There are', unknown_count, 'listings whose Location variable could not be defined.')
print('There are', unique_neighbourhood_count, 'different neighbourhoods')

There are 5688 listings whose Location variable could not be defined.
There are 92 different neighbourhoods


So there are still 5688 properties for which we could not extract the city. However, these are 92 districts. We will retrieve the city using GeoPy and the neighborhood names. We could find the locality based on latitude and longitude, however this would generate relatively significant calculation time and calls to an API. Furthermore, to convert the price of goods into euros when this is not the case (London or Prague), this would complicate the task (high number of localities). These data represent around 2,4% of the initial datasets, I decide to delete them.

In [18]:
# Remove rows where the 'Location' is 'Unknown'
data = data[data['Location'] != 'Unknown']

In [19]:
## Conversion of Price to Euro at the current exchange rate (26/11/2023)

# Exchange rates to Euro
exchange_rates_to_euro = {
    'Florence': 1,  # Already in Euro
    'Amsterdam': 1,  
    'Brussels': 1, 
    'Barcelona': 1,  
    'Lisbon': 1,  
    'Berlin': 1,  
    'Paris': 1,  
    'London': 1.15  # GBP to Euro
}
def convert_to_euro(row):
    rate = exchange_rates_to_euro.get(row['Location'], 1)
    return row['price'] / rate

data['price_in_euro'] = data.apply(convert_to_euro, axis=1)

In [20]:
# Vérification 
print(data.dtypes)

name                               object
host_id                             int64
host_name                          object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
Bedrooms                          float64
Beds                              float64
Baths                             float64
Rating                            float64
Housing types                      object
Property Type Cluster              object
Location                           object
price_in_euro                     float64
dtype: object


In [21]:
# Removing listings with outliers in price_in_euro
"""
Based on Location
I will remove entries with a price_in_euro < 5
I will use the first (Q1) and third (Q3) quartiles to calculate the Interquartile Range (IQR) and then filter out outliers from the data.
"""
data = data[data['price_in_euro'] > 5]

filtered_data = pd.DataFrame()

for location in data['Location'].unique():
    location_data = data[data['Location'] == location]

    Q1 = location_data['price_in_euro'].quantile(0.25)
    Q3 = location_data['price_in_euro'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    filtered_location_data = location_data[(location_data['price_in_euro'] >= lower_bound) & (location_data['price_in_euro'] <= upper_bound)]
    filtered_data = pd.concat([filtered_data, filtered_location_data])

data = filtered_data



In [22]:
# Removing listings with outliers in minimum_nights, considering that listings with more than 365 minimum nights are desactived. 
data = data[data['minimum_nights'] <= 365] 

In [23]:
data.describe()

,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,Bedrooms,Beds,Baths,Rating,price_in_euro
count,2.146530e+05,214653.000000,214653.000000,214653.000000,214653.000000,214653.000000,171151.000000,214653.000000,214653.000000,214653.000000,214653.000000,214653.000000,214653.000000,214653.000000,214653.000000
mean,1.511930e+08,48.624532,1.781124,129.791366,26.350319,31.109297,1.237140,15.141615,122.843855,8.851658,1.327002,1.420348,1.020221,3.043356,123.682092
std,1.686983e+08,4.189275,4.857850,82.909013,79.972521,68.871260,1.614001,49.735094,131.631831,18.016971,0.886678,0.817631,0.627047,2.255301,80.040224
min,1.581000e+03,38.679120,-9.488100,7.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.086957
25%,1.724000e+07,48.842306,-0.131437,68.000000,1.000000,1.000000,0.210000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,64.000000
50%,6.601603e+07,50.807670,2.170690,109.000000,3.000000,7.000000,0.690000,1.000000,68.000000,2.000000,1.000000,1.000000,1.000000,4.540000,103.000000
75%,2.593814e+08,51.521910,2.369630,174.000000,5.000000,28.000000,1.710000,5.000000,251.000000,10.000000,2.000000,2.000000,1.000000,4.840000,165.000000
max,5.369828e+08,52.656110,13.721390,525.000000,365.000000,3199.000000,120.110000,595.000000,365.000000,1689.000000,50.000000,50.000000,48.000000,5.000000,525.000000


In [24]:
# Visualization of categorical data
unique_values_dict = {}  # Dictionary to store unique values for each categorical column

# Loop through all columns to find unique values
for column in data.select_dtypes(include=['object']).columns:
    unique_values = data[column].unique()
    unique_values_dict[column] = unique_values

# Display the unique values for each categorical column
for key, value in unique_values_dict.items():
    print(f"Unique values in the column {key}: {value}")

Unique values in the column name: ['Serviced apartment in Florence · ★4.65 · 1 bedroom · 1 bed · 1 private bath'
 'Rental unit in Florence · ★4.88 · 1 bedroom · 1 bed · 1 bath'
 'Condo in Florence · ★4.77 · 2 bedrooms · 4 beds · 2 baths' ...
 'Rental unit in Gants Hill · ★New · 1 bedroom · 2 beds · 1 bath'
 'Barn in Greater London · ★New · 1 bedroom · 1 bed · 1 bath'
 'Home in Greater London · ★New · 1 bedroom · 5 beds · 2 shared baths']
Unique values in the column host_name: ['Officina 360' 'Lucia' 'Cesare' ... 'Sharmishta' 'Ana Gabriela' 'Kátia']
Unique values in the column neighbourhood: ['Centro Storico' 'Gavinana Galluzzo' 'Isolotto Legnaia' 'Campo di Marte'
 'Rifredi' 'Noord-Oost' 'Westerpark' 'Zuid' 'Centrum-West' 'Slotervaart'
 'Centrum-Oost' 'Oostelijk Havengebied - Indische Buurt'
 'Buitenveldert - Zuidas' 'IJburg - Zeeburgereiland' 'Oud-Oost'
 'De Pijp - Rivierenbuurt' 'De Baarsjes - Oud-West' 'Bos en Lommer'
 'Watergraafsmeer' 'Oud-Noord' 'Noord-West' 'Geuzenveld - Sloterme

In [25]:
data.to_csv('../data/data_augmented.csv')